In [13]:
%pip install tensorflow
%pip install keras
%pip install imblearn
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install tensorflow-addons
%pip install vit_keras
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 38.1/38.1 MB 20.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
import random

#   Plotting
import seaborn as sns
import matplotlib.pyplot as plt


#   DataGenerator to read images and rescale images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import tensorflow_addons as tfa

#   count each class samples
from collections import Counter

#   callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau

#   evaluate precison recall and f1-score of each class of model
from sklearn.metrics import classification_report
#   Show performance of a classification model
from sklearn.metrics import confusion_matrix

#   Different layers
from keras.initializers import GlorotUniform
from keras.models import Sequential
from keras.models import Model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Softmax
from tensorflow.keras.layers import GroupNormalization
from tensorflow.keras.layers import Attention

from vit_keras import vit, utils

# split dataset to train, validation and test set
from sklearn.model_selection import train_test_split

#   callbacks
from keras import callbacks

#   SMOTETomek from imblance library
from imblearn.combine import SMOTETomek

#   Optimizer
from tensorflow.keras.optimizers import SGD

from sklearn.metrics import roc_curve, auc
from itertools import cycle

In [15]:
## Set Path Here before running the code
WORKING_DIRECTORY =  "C:/Users/rahil/OneDrive/Desktop/Dataset"

##  Name of classes 
CLASSES = ['MildDemented',
           'ModerateDemented',
           'NonDemented',
           'VeryMildDemented']

X, y = [], []

## Images rescaling
datagen = ImageDataGenerator(rescale=1.0/255.0)

#   Load images by resizing and shuffling randomly
train_dataset = datagen.flow_from_directory(WORKING_DIRECTORY, target_size=(176, 208),batch_size=6400, shuffle=True)

### Seperate Dataset from  Data Genrator
X, y = train_dataset.next()

samples_before = len(X)
print("Images shape :\t", X.shape)
print("Labels shape :\t", y.shape)

Found 6400 images belonging to 4 classes.
Images shape :	 (6400, 176, 208, 3)
Labels shape :	 (6400, 4)


In [16]:
#   reshaping the images to 1D
X = X.reshape(-1, 176 * 208 * 3)

# create unbalanced dataset and dataset used in rebalancing
X, unbal_x, y, unbal_y = train_test_split(X, y, test_size=0.1)
unbal_x = unbal_x.reshape(-1, 176, 208, 3)

# unbalanced data
unbalanced_samples = len(unbal_x)
print("Number of samples in unbalanced dataset: \t", sorted(Counter(np.argmax(unbal_y, axis = 1)).items()))



Number of samples in unbalanced dataset: 	 [(0, 81), (1, 15), (2, 311), (3, 233)]


In [17]:
#   Oversampling method to remove imbalance class problem
X, y = SMOTETomek().fit_resample(X, y)

#   reshape images to images size of 208, 176, 3
X = X.reshape(-1, 176, 208, 3)

samples_after = len(X)
print("Number of samples after SMOTETomek :\t", sorted(Counter(np.argmax(y, axis=1)).items()))

Number of samples after SMOTETomek :	 [(0, 2889), (1, 2889), (2, 2889), (3, 2889)]


In [18]:
#   20% split to validation and 80% split to train set
X_train, x_val, y_train, y_val = train_test_split(X,y, test_size = 0.2)

#   20% split to test from 80% of train and 60% remains in train set
X_train, x_test, y_train, y_test = train_test_split(X_train,y_train, test_size = 0.2)

# Number of samples after train test split
print("Number of samples after splitting into Training, validation & test set\n")

print("Train     \t",sorted(Counter(np.argmax(y_train, axis=1)).items()))
print("Validation\t",sorted(Counter(np.argmax(y_val, axis=1)).items()))
print("Test      \t",sorted(Counter(np.argmax(y_test, axis=1)).items()))
print("Unbalanced\t",sorted(Counter(np.argmax(unbal_y, axis=1)).items()))

del X, y

Number of samples after splitting into Training, validation & test set

Train     	 [(0, 1868), (1, 1804), (2, 1819), (3, 1904)]
Validation	 [(0, 582), (1, 608), (2, 591), (3, 531)]
Test      	 [(0, 439), (1, 477), (2, 479), (3, 454)]
Unbalanced	 [(0, 81), (1, 15), (2, 311), (3, 233)]


In [ ]:

init = 'glorot_uniform'

# Define the ViT model
vit_model = vit.vit_b16(
    image_size=176,
    activation='softmax',
    pretrained=False,
    include_top=True,
    pretrained_top=False,
)

# Create a Sequential model and add the ViT model
RecallNET = Sequential()

# Add the ViT model as the first layer
RecallNET.add(vit_model)

# Add a GlobalAveragePooling2D layer if needed
RecallNET.add(GlobalAveragePooling2D())

# Add a Dense layer with the desired number of classes and softmax activation
RecallNET.add(Dense(4, activation='softmax', kernel_initializer=init))

# Display the model summary
RecallNET.summary()


In [ ]:
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation = tfa.activations.ReLU),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(64, activation = tfa.activations.gelu),
        tf.keras.layers.Dense(32, activation = tfa.activations.gelu),
        tf.keras.layers.Dense(3, 'softmax')
    ],
    name = 'vision_transformer')

model.summary()
